# Creating an External Solver Support Using Loandra 

# Fixed Height Tree Problem First 

In [2]:
import numpy as np
import pandas as pd
import openpyxl
from sklearn.preprocessing import LabelEncoder

from utils import * # Dataloader and K-fold mechanism 
from SATreeCraft import * # Tree solver framework
from SATreeClassifier import * 
import os
import subprocess
import loandra_support.loandra

In [3]:
from utils import *

file_path_to_test = 'Datasets/wine/wine.data'
delimiter = ','
label_position = 0 

data_loader = TreeDataLoaderBinaryNumerical(file_path=file_path_to_test, delimiter=delimiter, label_position= label_position)



print("Features:", data_loader.features, data_loader.features.shape)
print("Labels:", data_loader.labels, data_loader.labels.shape)
print("True Labels for Points:", data_loader.true_labels_for_points, data_loader.true_labels_for_points.shape)
print("Dataset:\n", data_loader.dataset,data_loader.dataset.shape)


features = data_loader.features
labels = data_loader.labels
true_labels_for_points = data_loader.true_labels_for_points
dataset = data_loader.dataset

Features: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12'] (13,)
Labels: [0 1 2] (3,)
True Labels for Points: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2] (178,)
Dataset:
 [[1.423e+01 1.710e+00 2.430e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [1.327e+01 4.280e+00 2.260e+00 ... 5.900e-01 1.560e+00 8.350e+02]
 [1.317e+01 2.590e+00 2.370e+00 ... 6.000e-01 1.620e+00 8.400e+02]
 [1.413e+01 4.100e+00 2.740e+00 ... 6.100e-01 1.600e+00 5.600e+02]] (178, 13)


In [4]:
max_accuracy_numerical_problem = SATreeCraft(dataset=dataset,
                                             features=features,labels=labels,
                                             true_labels_for_points=true_labels_for_points,
                                             classification_objective='max_accuracy',
                                             fixed_depth=2,
                                             # min_support= 5
                                             # min_margin  = 10
                                             # tree_structure= 'Oblivious'
                                             )
#max_accuracy_numerical_problem.solve()

In [5]:
loandra_path = '/Users/harisrasul/Desktop/loandra'
max_accuracy_numerical_problem.solve_loandra(loandra_path= loandra_path)


In [6]:
print("Final Model: ", max_accuracy_numerical_problem.model)
print("Min cost found: ", max_accuracy_numerical_problem.min_cost)

Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '6', 'threshold': 1.885}, {'type': 'branching', 'children': [3, 4], 'feature': '9', 'threshold': 3.82}, {'type': 'branching', 'children': [5, 6], 'feature': '12', 'threshold': 730.0}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 2}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 0}]
Min cost found:  6


In [7]:
# Classifier - sklearn Integration 

# Build model
model = SATreeClassifier(max_accuracy_numerical_problem.model)

# Test cases 0- uisng training set expecting 100%
X_test = dataset[5:160]
y_test = true_labels_for_points[5:160]

# To get the accuracy score
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

# To get the classification report
report = model.get_classification_report(X_test, y_test)
print("Classification Report:")
print(report)

# To get the confusion matrix
conf_matrix = model.get_confusion_matrix(X_test, y_test)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9612903225806452
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.96        54
           1       0.99      0.93      0.96        71
           2       0.94      1.00      0.97        30

    accuracy                           0.96       155
   macro avg       0.96      0.97      0.96       155
weighted avg       0.96      0.96      0.96       155

Confusion Matrix:
[[53  1  0]
 [ 3 66  2]
 [ 0  0 30]]


In [8]:
k = 5
depth = 2 
minimum_support = 5
accuracies, mean_score = k_fold_tester(k=k, depth=depth, dataset=dataset, true_labels_for_points=true_labels_for_points, 
                                       labels=labels, features=features, min_support_level=minimum_support, 
                                       # complete_tree=False
                                       )

print(accuracies)
print(mean_score)

Iteration complete
Iteration complete
Iteration complete
Iteration complete
Iteration complete
[0.94444444 0.94444444 0.94444444 0.94285714 0.94285714]
0.9438095238095239


In [10]:
k = 5
depth = 2 
minimum_support = 5
loandra_path = '/Users/harisrasul/Desktop/loandra'

accuracies, mean_score = k_fold_tester_loandra(loandra_path=loandra_path,
                                               k=k, depth=depth, dataset=dataset, true_labels_for_points=true_labels_for_points, 
                                       labels=labels, features=features, min_support_level=minimum_support, 
                                       # complete_tree=False
                                       )

print(accuracies)
print(mean_score)

Iteration complete
Iteration complete
Iteration complete
Iteration complete
Iteration complete
[0.97222222 0.94444444 0.86111111 0.97142857 0.91428571]
0.9326984126984128


Categorical Testing for fixed height

In [11]:
# Promoter data set
file_path = 'Datasets/molecular+biology+promoter+gene+sequences/promoters.data'
label_index = 0
categorical_feature_index = 2  
numerical_indices = None

data_loader = TreeDataLoaderWithCategorical(
    file_path= file_path,
    label_index= label_index,
    numerical_indices= numerical_indices,
    categorical_feature_index=categorical_feature_index
)

print("Features:", data_loader.features, data_loader.features.shape)
print("Categorical Features:", data_loader.features_categorical, data_loader.features_categorical.shape)
print("Numerical Features:", data_loader.features_numerical, data_loader.features_numerical.shape)
print("Labels:", data_loader.labels, data_loader.labels.shape)
print("True Labels for Points:", data_loader.true_labels_for_points, data_loader.true_labels_for_points.shape)
print("Dataset:\n", data_loader.dataset,data_loader.dataset.shape)


features = data_loader.features
features_categorical = data_loader.features_categorical
features_numerical = data_loader.features_numerical
labels = data_loader.labels
true_labels_for_points = data_loader.true_labels_for_points
dataset = data_loader.dataset

Features: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56'] (57,)
Categorical Features: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56'] (57,)
Numerical Features: [] (106, 0)
Labels: [0 1] (2,)
True Labels for Points: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] (106,)
Dataset:
 [['t' 'a' 'c' ... 'c' 'g' 't']
 ['t' 'g' 'c' ... 'c' 'a' 'a']
 ['g' 't' 'a' ... 'g' 'c' 

In [12]:
max_accuracy_categorical_problem = SATreeCraft(dataset=dataset,
                                           features=features,
                                           labels=labels,
                                           true_labels_for_points=true_labels_for_points,
                                           features_categorical=features_categorical,
                                           features_numerical=features_numerical,
                                           classification_objective= 'max_accuracy',
                                           fixed_depth=2,
                                           # min_support= 2,
                                           # tree_structure = 'Oblivious'
                                           )

loandra_path = '/Users/harisrasul/Desktop/loandra'
max_accuracy_categorical_problem.solve_loandra(loandra_path)
print("Final Model: ", max_accuracy_categorical_problem.model)
print("Min cost found: ", max_accuracy_categorical_problem.min_cost)

/Users/harisrasul/Desktop/ESC499-THESIS/ESC499-Thesis-SAT-Trees/SATreeCraft/classification_problems/fixed_height_tree_categorical_module.py:110: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if str(j) in features_numerical:


Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '38', 'threshold': ['a', 't']}, {'type': 'branching', 'children': [3, 4], 'feature': '14', 'threshold': ['a', 'c']}, {'type': 'branching', 'children': [5, 6], 'feature': '9', 'threshold': ['a']}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 0}, {'type': 'leaf', 'label': 0}, {'type': 'leaf', 'label': 1}]
Min cost found:  12


In [13]:
max_accuracy_categorical_problem = SATreeCraft(dataset=dataset,
                                           features=features,
                                           labels=labels,
                                           true_labels_for_points=true_labels_for_points,
                                           features_categorical=features_categorical,
                                           features_numerical=features_numerical,
                                           classification_objective= 'max_accuracy',
                                           fixed_depth=2,
                                           # min_support= 2,
                                           # tree_structure = 'Oblivious'
                                           )


max_accuracy_categorical_problem.solve()
print("Final Model: ", max_accuracy_categorical_problem.model)
print("Min cost found: ", max_accuracy_categorical_problem.min_cost)

/Users/harisrasul/Desktop/ESC499-THESIS/ESC499-Thesis-SAT-Trees/SATreeCraft/classification_problems/fixed_height_tree_categorical_module.py:110: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if str(j) in features_numerical:


Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '38', 'threshold': ['a', 't']}, {'type': 'branching', 'children': [3, 4], 'feature': '14', 'threshold': ['a', 'c']}, {'type': 'branching', 'children': [5, 6], 'feature': '9', 'threshold': ['a']}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 0}, {'type': 'leaf', 'label': 0}, {'type': 'leaf', 'label': 1}]
Min cost found:  12


# Minimum Height Problem With Loandra

In [14]:
file_path_to_test = 'Datasets/wine/wine.data'
delimiter = ','
label_position = 0 

data_loader = TreeDataLoaderBinaryNumerical(file_path=file_path_to_test, delimiter=delimiter, label_position= label_position)



print("Features:", data_loader.features, data_loader.features.shape)
print("Labels:", data_loader.labels, data_loader.labels.shape)
print("True Labels for Points:", data_loader.true_labels_for_points, data_loader.true_labels_for_points.shape)
print("Dataset:\n", data_loader.dataset,data_loader.dataset.shape)


features = data_loader.features
labels = data_loader.labels
true_labels_for_points = data_loader.true_labels_for_points
dataset = data_loader.dataset

Features: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12'] (13,)
Labels: [0 1 2] (3,)
True Labels for Points: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2] (178,)
Dataset:
 [[1.423e+01 1.710e+00 2.430e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [1.327e+01 4.280e+00 2.260e+00 ... 5.900e-01 1.560e+00 8.350e+02]
 [1.317e+01 2.590e+00 2.370e+00 ... 6.000e-01 1.620e+00 8.400e+02]
 [1.413e+01 4.100e+00 2.740e+00 ... 6.100e-01 1.600e+00 5.600e+02]] (178, 13)


In [15]:
min_height_numerical_problem = SATreeCraft(dataset=dataset,features=features,labels=labels,true_labels_for_points=true_labels_for_points)
min_height_numerical_problem.solve()

print("Final Model: ", min_height_numerical_problem.model)
print("min depth found: ", min_height_numerical_problem.min_depth)

no solution at depth 1
no solution at depth 2
Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '10', 'threshold': 0.915}, {'type': 'branching', 'children': [3, 4], 'feature': '6', 'threshold': 1.585}, {'type': 'branching', 'children': [5, 6], 'feature': '12', 'threshold': 716.0}, {'type': 'branching', 'children': [7, 8], 'feature': '9', 'threshold': 3.8449999999999998}, {'type': 'branching', 'children': [9, 10], 'feature': '12', 'threshold': 627.5}, {'type': 'branching', 'children': [11, 12], 'feature': '6', 'threshold': 0.565}, {'type': 'branching', 'children': [13, 14], 'feature': '9', 'threshold': 3.46}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 2}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 0}, {'type': 'leaf', 'label': 2}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 0}]
min depth found:  3


In [16]:
loandra_path = '/Users/harisrasul/Desktop/loandra'

min_height_numerical_problem = SATreeCraft(dataset=dataset,features=features,labels=labels,true_labels_for_points=true_labels_for_points)
min_height_numerical_problem.solve_loandra(loandra_path)
print("Final Model: ", min_height_numerical_problem.model)
print("min depth found: ", min_height_numerical_problem.min_depth)

no solution at depth 1
no solution at depth 2
Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '9', 'threshold': 3.915}, {'type': 'branching', 'children': [3, 4], 'feature': '12', 'threshold': 765.0}, {'type': 'branching', 'children': [5, 6], 'feature': '6', 'threshold': 1.5750000000000002}, {'type': 'branching', 'children': [7, 8], 'feature': '6', 'threshold': 0.495}, {'type': 'branching', 'children': [9, 10], 'feature': '0', 'threshold': 13.04}, {'type': 'branching', 'children': [11, 12], 'feature': '3', 'threshold': 16.9}, {'type': 'branching', 'children': [13, 14], 'feature': '12', 'threshold': 679.0}, {'type': 'leaf', 'label': 2}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 0}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 2}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 0}]
min depth found:  3


Categorical Problem 

In [17]:
# Promoter data set
file_path = 'Datasets/molecular+biology+promoter+gene+sequences/promoters.data'
label_index = 0
categorical_feature_index = 2  
numerical_indices = None

data_loader = TreeDataLoaderWithCategorical(
    file_path= file_path,
    label_index= label_index,
    numerical_indices= numerical_indices,
    categorical_feature_index=categorical_feature_index
)

print("Features:", data_loader.features, data_loader.features.shape)
print("Categorical Features:", data_loader.features_categorical, data_loader.features_categorical.shape)
print("Numerical Features:", data_loader.features_numerical, data_loader.features_numerical.shape)
print("Labels:", data_loader.labels, data_loader.labels.shape)
print("True Labels for Points:", data_loader.true_labels_for_points, data_loader.true_labels_for_points.shape)
print("Dataset:\n", data_loader.dataset,data_loader.dataset.shape)


features = data_loader.features
features_categorical = data_loader.features_categorical
features_numerical = data_loader.features_numerical
labels = data_loader.labels
true_labels_for_points = data_loader.true_labels_for_points
dataset = data_loader.dataset

Features: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56'] (57,)
Categorical Features: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56'] (57,)
Numerical Features: [] (106, 0)
Labels: [0 1] (2,)
True Labels for Points: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] (106,)
Dataset:
 [['t' 'a' 'c' ... 'c' 'g' 't']
 ['t' 'g' 'c' ... 'c' 'a' 'a']
 ['g' 't' 'a' ... 'g' 'c' 

In [18]:
min_height_categorical_problem = SATreeCraft(dataset=dataset,
                                           features=features,
                                           labels=labels,
                                           true_labels_for_points=true_labels_for_points,
                                           features_categorical=features_categorical,
                                           features_numerical=features_numerical)
min_height_categorical_problem.solve()


print("Final Model: ", min_height_categorical_problem.model)
print("min depth found: ", min_height_categorical_problem.min_depth)

/Users/harisrasul/Desktop/ESC499-THESIS/ESC499-Thesis-SAT-Trees/SATreeCraft/classification_problems/min_height_tree_categorical_module.py:124: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if str(j) in features_numerical:


No solution at depth:  1
No solution at depth:  2
No solution at depth:  3
Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '28', 'threshold': ['a', 'c']}, {'type': 'branching', 'children': [3, 4], 'feature': '1', 'threshold': ['a', 'g', 't']}, {'type': 'branching', 'children': [5, 6], 'feature': '54', 'threshold': ['a', 't']}, {'type': 'branching', 'children': [7, 8], 'feature': '29', 'threshold': ['a']}, {'type': 'branching', 'children': [9, 10], 'feature': '49', 'threshold': ['a', 'g']}, {'type': 'branching', 'children': [11, 12], 'feature': '16', 'threshold': ['a', 't']}, {'type': 'branching', 'children': [13, 14], 'feature': '35', 'threshold': ['a', 'g']}, {'type': 'branching', 'children': [15, 16], 'feature': '38', 'threshold': ['a', 't']}, {'type': 'branching', 'children': [17, 18], 'feature': '16', 'threshold': ['a', 'c', 't']}, {'type': 'branching', 'children': [19, 20], 'feature': '41', 'threshold': ['a']}, {'type': 'branching', 'children': [21, 22], 'featu

In [19]:
loandra_path = '/Users/harisrasul/Desktop/loandra'

min_height_categorical_problem = SATreeCraft(dataset=dataset,
                                           features=features,
                                           labels=labels,
                                           true_labels_for_points=true_labels_for_points,
                                           features_categorical=features_categorical,
                                           features_numerical=features_numerical)

min_height_categorical_problem.solve_loandra(loandra_path)

print("Final Model: ", min_height_categorical_problem.model)
print("min depth found: ", min_height_categorical_problem.min_depth)

/Users/harisrasul/Desktop/ESC499-THESIS/ESC499-Thesis-SAT-Trees/SATreeCraft/classification_problems/min_height_tree_categorical_module.py:124: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if str(j) in features_numerical:


No solution at depth:  1
No solution at depth:  2
No solution at depth:  3
Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '43', 'threshold': ['a', 'g']}, {'type': 'branching', 'children': [3, 4], 'feature': '2', 'threshold': ['a', 'c', 't']}, {'type': 'branching', 'children': [5, 6], 'feature': '3', 'threshold': ['a', 'g']}, {'type': 'branching', 'children': [7, 8], 'feature': '14', 'threshold': ['a', 'c']}, {'type': 'branching', 'children': [9, 10], 'feature': '47', 'threshold': ['a', 'g']}, {'type': 'branching', 'children': [11, 12], 'feature': '35', 'threshold': ['a', 'c']}, {'type': 'branching', 'children': [13, 14], 'feature': '40', 'threshold': ['a', 'g']}, {'type': 'branching', 'children': [15, 16], 'feature': '19', 'threshold': ['a', 'g', 't']}, {'type': 'branching', 'children': [17, 18], 'feature': '38', 'threshold': ['a', 't']}, {'type': 'branching', 'children': [19, 20], 'feature': '19', 'threshold': ['a', 't']}, {'type': 'branching', 'children': [21, 22

In [20]:
# Classifier - sklearn Integration 

# Build model
model = SATreeClassifier(min_height_categorical_problem.model)

# Test cases 0- uisng training set expecting 100%
X_test = dataset
y_test = true_labels_for_points

# To get the accuracy score
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

# To get the classification report
report = model.get_classification_report(X_test, y_test)
print("Classification Report:")
print(report)

# To get the confusion matrix
conf_matrix = model.get_confusion_matrix(X_test, y_test)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        53
           1       1.00      1.00      1.00        53

    accuracy                           1.00       106
   macro avg       1.00      1.00      1.00       106
weighted avg       1.00      1.00      1.00       106

Confusion Matrix:
[[53  0]
 [ 0 53]]


Worst Problem Changes

In [21]:
# Promoter data set
file_path_to_test = 'Datasets/vertebral+column/verbex.data'
delimiter = ','
label_position = -1 

data_loader = TreeDataLoaderBinaryNumerical(file_path=file_path_to_test, delimiter=delimiter, label_position= label_position)



print("Features:", data_loader.features, data_loader.features.shape)
print("Labels:", data_loader.labels, data_loader.labels.shape)
print("True Labels for Points:", data_loader.true_labels_for_points, data_loader.true_labels_for_points.shape)
print("Dataset:\n", data_loader.dataset,data_loader.dataset.shape)


features = data_loader.features
labels = data_loader.labels
true_labels_for_points = data_loader.true_labels_for_points
dataset = data_loader.dataset

Features: ['0' '1' '2' '3' '4' '5'] (6,)
Labels: [0 1] (2,)
True Labels for Points: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1] (310,)
Dataset:
 [[ 63.0278175   22.55258597  39.60911701  40.47523153  98.67291675
   -0.25439999]
 [ 39.05695098  10.06099147  25.01537822  28.99595951 114.4054254
    4.56425864]
 [ 68.83202098  22.21848205  50.09219357  46.61353893 105.9851355
   -3.53031731]
 ...
 [ 61.44659663  2

Min Height

In [22]:
min_height_numerical_problem = SATreeCraft(dataset=dataset,features=features,labels=labels,true_labels_for_points=true_labels_for_points)
min_height_numerical_problem.solve()

print("Final Model: ", min_height_numerical_problem.model)
print("min depth found: ", min_height_numerical_problem.min_depth)

no solution at depth 1
no solution at depth 2
no solution at depth 3
no solution at depth 4
Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '4', 'threshold': 118.35335505}, {'type': 'branching', 'children': [3, 4], 'feature': '5', 'threshold': 1.88035368}, {'type': 'branching', 'children': [5, 6], 'feature': '1', 'threshold': 15.00648001}, {'type': 'branching', 'children': [7, 8], 'feature': '5', 'threshold': -1.151194007}, {'type': 'branching', 'children': [9, 10], 'feature': '3', 'threshold': 37.838869364999994}, {'type': 'branching', 'children': [11, 12], 'feature': '1', 'threshold': 12.523317200000001}, {'type': 'branching', 'children': [13, 14], 'feature': '0', 'threshold': 54.920150055}, {'type': 'branching', 'children': [15, 16], 'feature': '5', 'threshold': -4.164346918}, {'type': 'branching', 'children': [17, 18], 'feature': '4', 'threshold': 112.49681615}, {'type': 'branching', 'children': [19, 20], 'feature': '4', 'threshold': 115.84036425}, {'type': 'bra

In [23]:
loandra_path = '/Users/harisrasul/Desktop/loandra'

min_height_numerical_problem = SATreeCraft(dataset=dataset,features=features,labels=labels,true_labels_for_points=true_labels_for_points)

min_height_numerical_problem.solve_loandra(loandra_path)

print("Final Model: ", min_height_numerical_problem.model)
print("min depth found: ", min_height_numerical_problem.min_depth)

no solution at depth 1
no solution at depth 2
no solution at depth 3
no solution at depth 4
Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '4', 'threshold': 120.47806725000001}, {'type': 'branching', 'children': [3, 4], 'feature': '1', 'threshold': 17.938592895}, {'type': 'branching', 'children': [5, 6], 'feature': '3', 'threshold': 29.929307225000002}, {'type': 'branching', 'children': [7, 8], 'feature': '4', 'threshold': 116.40843505000001}, {'type': 'branching', 'children': [9, 10], 'feature': '5', 'threshold': 1.6787053455}, {'type': 'branching', 'children': [11, 12], 'feature': '1', 'threshold': 16.35768863}, {'type': 'branching', 'children': [13, 14], 'feature': '2', 'threshold': 46.901048125}, {'type': 'branching', 'children': [15, 16], 'feature': '5', 'threshold': 5.970002535000001}, {'type': 'branching', 'children': [17, 18], 'feature': '5', 'threshold': 9.209473302500001}, {'type': 'branching', 'children': [19, 20], 'feature': '2', 'threshold': 50.9794820

In [24]:
model = SATreeClassifier(min_height_numerical_problem.model)

# Test cases 0- uisng training set expecting 100%
X_test = dataset
y_test = true_labels_for_points

# To get the accuracy score
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

# To get the classification report
report = model.get_classification_report(X_test, y_test)
print("Classification Report:")
print(report)

# To get the confusion matrix
conf_matrix = model.get_confusion_matrix(X_test, y_test)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       210
           1       1.00      1.00      1.00       100

    accuracy                           1.00       310
   macro avg       1.00      1.00      1.00       310
weighted avg       1.00      1.00      1.00       310

Confusion Matrix:
[[210   0]
 [  0 100]]


Max Accuracy

In [25]:
max_accuracy_numerical_problem = SATreeCraft(dataset=dataset,
                                             features=features,labels=labels,
                                             true_labels_for_points=true_labels_for_points,
                                             classification_objective='max_accuracy',
                                             fixed_depth=2,
                                             # min_support= 5
                                             # min_margin  = 10
                                             # tree_structure= 'Oblivious'
                                             )
max_accuracy_numerical_problem.solve()
print("Final Model: ", max_accuracy_numerical_problem.model)
print("Min cost found: ", max_accuracy_numerical_problem.min_cost)

Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '5', 'threshold': 5.20209819}, {'type': 'branching', 'children': [3, 4], 'feature': '3', 'threshold': 28.136471795}, {'type': 'branching', 'children': [5, 6], 'feature': '0', 'threshold': 41.179724805}, {'type': 'leaf', 'label': 0}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 0}]
Min cost found:  45


In [26]:
max_accuracy_numerical_problem = SATreeCraft(dataset=dataset,
                                             features=features,labels=labels,
                                             true_labels_for_points=true_labels_for_points,
                                             classification_objective='max_accuracy',
                                             fixed_depth=2,
                                             # min_support= 5
                                             # min_margin  = 10
                                             # tree_structure= 'Oblivious'
                                             )
loandra_path = '/Users/harisrasul/Desktop/loandra'

max_accuracy_numerical_problem.solve_loandra(loandra_path)
print("Final Model: ", max_accuracy_numerical_problem.model)
print("Min cost found: ", max_accuracy_numerical_problem.min_cost)

Final Model:  [{'type': 'branching', 'children': [1, 2], 'feature': '5', 'threshold': 6.3625154935}, {'type': 'branching', 'children': [3, 4], 'feature': '3', 'threshold': 28.136471795}, {'type': 'branching', 'children': [5, 6], 'feature': '2', 'threshold': 24.49754631}, {'type': 'leaf', 'label': 0}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 1}, {'type': 'leaf', 'label': 0}]
Min cost found:  45


In [27]:
model = SATreeClassifier(max_accuracy_numerical_problem.model)

# Test cases 0- uisng training set expecting 100%
X_test = dataset
y_test = true_labels_for_points

# To get the accuracy score
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

# To get the classification report
report = model.get_classification_report(X_test, y_test)
print("Classification Report:")
print(report)

# To get the confusion matrix
conf_matrix = model.get_confusion_matrix(X_test, y_test)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8548387096774194
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       210
           1       0.76      0.80      0.78       100

    accuracy                           0.85       310
   macro avg       0.83      0.84      0.84       310
weighted avg       0.86      0.85      0.86       310

Confusion Matrix:
[[185  25]
 [ 20  80]]
